# AI in Games, _Reinforcement Learning_<br>Assignment 2, Question 2:<br>**Tabular Model-based Methods**

## Introduction to the concept
Model-based methods are algorithms that use (thus require) a known environment model. "Tabular" refers to the use of look-up tables or arrays that store previous estimates of a value function (ex. state value function) and use these estimates to update them every iteration. It is not surprising that when the environment model is known, i.e. when the state-transition probabilities and rewards are known for every state-action pair, the evaluation an improvement of policies becomes much more efficient.

## Preparing the context
The following are the necessary preparations and imports needed to run and test the main code of this document in the intended context. Mounting directory & setting present working directory...

In [ ]:
# Mounting the Google Drive folder (run if necessary):
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
# Saving the present working directory's path:
pwd = "./drive/MyDrive/ColabNotebooks/AIG-Labs/AIG-Assignment2/"

Mounted at /content/drive/


To install module `import_ipynb` to enable importing Jupyter Notebooks as modules...

`!pip install import_ipynb`

Importing the code in notebook `Q1_environment.ipynb`...



In [ ]:
import import_ipynb
import numpy as np
N = import_ipynb.NotebookLoader(path=[pwd])
N.load_module("Q1_environment")
from Q1_environment import *

importing Jupyter notebook from ./drive/MyDrive/ColabNotebooks/AIG-Labs/AIG-Assignment2/Q1_environment.ipynb


## Policy evaluation
This function estimates the state value function (represented by an array of values per state) give a certain policy. Its goal is not to modify the policy but simply try to accurately obtain the value of each state. The value of a state is updated by the formula:

$\forall s \in S, V(s) \leftarrow V(s) + \sum_{a \in A} \pi(s, a) \sum_{s' \in S} P(s'|s, a) (R(s'|s, a) + \gamma V(s'))$

Here, we have the following:

- $S$: The set of all possible states
- $A$: The set of all possible actions
- $V$: The estimated value function
- $\pi$: The policy, which maps $s \in S$ & $a \in A$ to the probability of taking $a$ from $s$
- $P(s'|s, a)$: The probability of transitioning to $s'$ having taking $a$ from $s$
- $R(s'|s, a)$: The reward of transitioning to $s'$ having taking $a$ from $s$
- $\gamma$: The discount factor (a constant predefined parameter)

<br>The policy evaluation function is designed to ensure that:<br>$\forall s \in S, V(s) \rightarrow V^{\pi}(s)$ as $i \rightarrow \infty$

Here, $V^{\pi}$ is the true value function for policy $\pi$ & $i$ is the number of iterations of the algorithm.

In [ ]:
def policy_evaluation(env, policy, gamma, theta, max_iterations):
    '''
    NOTE ON THE ARGUMENTS:
    `env`: Object of the chosen environment class (ex. FrozenLake)
    `policy`: Array giving the probability of taking an action from a state
    `gamma`: Discount factor
    `theta`: Error tolerance level

    FURTHER NOTE ON `policy`:
    We consider the policy to be deterministic, meaning that it maps each state
    to a certain action, rather than each state-action pair to a probability.
    Hence, `policy` is a 1D array with each index corresponding to a state and
    the value at a given index i corresponding to the action to be taken from
    state i. This is equivalent to the policy wherein each state-action pair is
    related to either 0 or 1 so that each state is mapped to 1 only when paired
    with a particular action.
    '''
    # Initialising table of values per state:
    value = np.zeros(env.n_states, dtype=float)
    # Flag for indicating convergence of value evaluation:
    flag = 0
    # Policy evaluation loop:
    for i in range(max_iterations):
        for s in range(env.n_states):
            # NOTE: s ==> state
            # Storing previous value of value function:
            v = value[s]
            #------------------------------------
            # Obtaining current value of value function:
            # NOTE: We iterate through every possible action from state `s`
            value[s] = 0
            for a in range(env.n_actions):
                # NOTE: a ==> action
                # If policy does not map `s` to `a`, move to next action:
                if policy[s] != a: continue
                # If policy does map `s` to `a`, update state value:
                for _s in range(env.n_states):
                    # NOTE: _s ==> next state
                    value[s] += env.p(_s,s,a)*(env.r(_s,s,a) + gamma*value[_s])
                    '''
                    NOTE ON ABOVE USED FUNCTIONS:
                    env.p: Probability of moving from s to _s given action a
                    env.r: Reward of moving from s to _s given action a
                    '''
            #------------------------------------
            # Obtaining the difference in state value:
            # NOTE: This is why we stored the previous value before in `v`
            if abs(value[s]-v) < theta: flag += 1

        # If difference in state value < theta for all states, stop:
        if flag == env.n_states: break
    return value

## Policy improvement

In [ ]:
def policy_improvement(env, value, gamma):
    '''
    NOTE ON THE ARGUMENTS:
    `env`: Object of the chosen environment class (ex. FrozenLake)
    `value`: Array containing values of each state with respect to some policy
    `gamma`: Discount factor

    NOTE ON POLICY IMPROVEMENT:
    The goal of policy improvement is to improve on the previously used policy
    (which is implicit in the array of state values, which is evaluated with
    respect to some policy). We do this by choosing for each state s the action
    a such that we maximise the reward of taking a from s (irrespective of
    policy) then following the previous policy (which is implicit in the array
    of state values).
    '''
    policy = np.zeros(env.n_states, dtype=int)
    for s in range(env.n_states):
        q = np.zeros(env.n_actions, dtype=np.float32)
        for a in range(env.n_actions):
            for _s in range(env.n_states):
                # NOTE: _s ==> next state
                # Total reward of taking a from s then following last policy:
                '''
                NOTE ON LAST POLICY:
                The previous policy based on which we are making the current
                improvement is implicit in the array of state values `value`,
                since this array was obtained with respect to some policy.
                '''
                q[a] += env.p(_s, s, a)*(env.r(_s, s, a) + gamma*value[_s])
                '''
                NOTE ON ABOVE USED FUNCTIONS:
                env.p: Probability of moving from s to _s given action a
                env.r: Reward of moving from s to _s given action a
                '''
        # Update policy to maximise the one-step dynamics from s:
        policy[s] = np.argmax(q)
    return policy

Policy iteration...

In [ ]:
def policy_iteration(env, gamma, theta, max_iterations, policy=None):
    if policy is None: policy = np.zeros(env.n_states, dtype=int)
    else: policy = np.array(policy, dtype=int)
    # Initialising state values with respect to existing policy:
    value = policy_evaluation(env, policy, gamma, theta, max_iterations)
    # Policy iteration loop:
    for i in range(max_iterations):
        policy = policy_improvement(env, value, gamma)
        new_value = policy_evaluation(env, policy, gamma, theta, max_iterations)
        # If all value evaluations change less than theta, break:
        if all(abs(new_value-value) < theta): break
        # Else, continue improving with the newly evaluated state values:
        value = new_value
    return policy, value

## Value iteration

In [ ]:
def value_iteration(env, gamma, theta, max_iterations, value=None):
    if value is None: value = np.zeros(env.n_states)
    else: value = np.array(value, dtype=np.float)
    # Flag for indicating convergence of value evaluation:
    flag = 0
    # Value iteration loop:
    for i in range(max_iterations):
        for s in range(env.n_states):
            q = np.zeros(env.n_actions, dtype=np.float32)
            for a in range(env.n_actions):
                for _s in range(env.n_states):
                    # NOTE: _s ==> next state
                    # Total reward of taking a from s for previous state value:
                    q[a] += env.p(_s, s, a)*(env.r(_s, s, a) + gamma*value[_s])
                    '''
                    NOTE ON ABOVE USED FUNCTIONS:
                    env.p: Probability of moving from s to _s given action a
                    env.r: Reward of moving from s to _s given action a
                    '''
            # Update policy to maximise the one-step dynamics from s:
            v = value[s]
            value[s] = np.max(q)
            if abs(value[s]-v) < theta: flag += 1

        # If difference in state value < theta for all states, stop:
        if flag == env.n_states: break

    #================================================

    # Obtaining the (estimated) optimal policy:
    # NOTE: The logic for this is identical to policy improvement
    policy = policy_improvement(env, value, gamma)
    return policy, value

## Testing the above functions
_The function testing code must not run if this file is imported as a module, hence we do..._<br>`if __name__ == '__main__'`<br>_... to check if the current file is being executed as the main code._

In [ ]:
if __name__ == '__main__':
    # Defining the parameters:
    env = FrozenLake(lake['small'], 0.1, 100)
    gamma = 0.9
    theta = 0.01
    max_iterations = 100

    # Running the functions:
    PI = policy_iteration(env, gamma, theta, max_iterations)
    VI = value_iteration(env, gamma, theta, max_iterations)
    labels = ("policy iteration", "value iteration")

    # Displaying results:
    displayResults((PI, VI), labels, env)



AGENT PERFORMANCE AFTER POLICY ITERATION

Lake:
[['&' '.' '.' '.']
 ['.' '#' '.' '#']
 ['.' '.' '.' '#']
 ['#' '.' '.' '$']]
Policy:
[['_' '>' '_' '<']
 ['_' '^' '_' '^']
 ['>' '_' '_' '^']
 ['^' '>' '>' '^']]
Value:
[[0.531 0.590 0.656 0.590]
 [0.590 0.000 0.729 0.000]
 [0.656 0.729 0.810 0.000]
 [0.000 0.810 0.900 1.000]]


AGENT PERFORMANCE AFTER VALUE ITERATION

Lake:
[['&' '.' '.' '.']
 ['.' '#' '.' '#']
 ['.' '.' '.' '#']
 ['#' '.' '.' '$']]
Policy:
[['_' '>' '_' '<']
 ['_' '^' '_' '^']
 ['>' '_' '_' '^']
 ['^' '>' '>' '^']]
Value:
[[0.531 0.590 0.656 0.590]
 [0.590 0.000 0.729 0.000]
 [0.656 0.729 0.810 0.000]
 [0.000 0.810 0.900 1.000]]
